#  Requesting Portfolio Data #

- **Initial Approach:**
  
  * Started off by using ```MultiIndex()``` as a way to group stocks and options
  * The point of using ```MultiIndex()``` was to help in determining total equities value and total options value and organize the data
  * I tried to do everything in one dataframe but I didn't get the results the way I wanted
  
- **Improved Approach:**
  
   * ✅Step 1: Create table for Total Equities Market Value , Total Options Market Value , Total Market Value, Mkt Val %
       * ```MulitIndex()``` is only useful for formating the way the dataframe is printed,I played around with it a lot but didn't really get the desired results, it is better to use ```groupby()``` and ```getgroup()``` in conjunction with ```MultiIndex()```
       * Once grouped into Stocks and Options, split the dataframe in two, one for stocks and the other for options:
         * Sum the market values of each stock / option to determine the Total Equities/Options Market Value
         * Determine percentages
         * Append results to each dataframe
      * Create dataframe for total market values
  
  * ✅❗Step 2: Add **Post Expiry Market Value** and **Post Expiry Market Value %**
      
      * ```portfolio()``` returned a dataframe which had all the required columns, they just had different names such as
         * **'averageCost'** is the same as **'Cost Price'**
         * **'unrealizedPNL'** is the same as **'P&L'**
         * **'position'** is the same as **'Qty'**
      * The columns were renamed and rearranged
      * Additional columns / rows that were added:
         * **'% Market Value'**  for stocks protfolio
             * 🆕✅The denominator is changed to `netLiquidationValue` which we get from `accountSummary`
         * **'Totals'** row  which displays **'Total Market Value'**, **'Total % Market Value'** and **'Total P&L'** for each portfolio
         * **'Type','Stirke', 'Underlying Equity Market Price', 'Total Market Value Post Expiry'** for options portfolio
      * ❗ For **'Underlying Equity Market Price'**:
         * I use the symbol / Ticker to create a contract
         * Qualify the contract
         * Get ticker for the contract, takes 11 seconds
         * Get market price which will be used as **'Underlying Equity Market Price'**
      * For **'Post Expiry Market Value'**:
         * I am creating creating the required conditions as shown in sample portfolio excel
         * Based on my understanding if it is a Put, post expiry market is calculated as follows:
             * *(Strike - Underlying Equity) * Quantity * 100*
         * If it is a Call,post expiry market is calculated as follows:
             * *(Underlying Equity - Strike) * Quantity * 100*
         * ✅ If **expired**, then post expiry market value is 0
         * ✅If it is a Call and underlying equity is less than strike then post expiry market value is zero
         * ✅If it is a Put and underlying equity is greater than strike then post expiry market value is zero
  
  * ✅Step 3: Create table for totals
      * ✅ Process dataframes for stocks and options
         
- **Problems:**
    * For **'Underlying Equity Market Price'**, I understand that it is the Market Price of the equity, however I am not sure if I should extract it from the stocks portfolio or create a contract, get its ticker and then the market price
    * ✅ Options have an attribute ```lastTradeDateOrContractMonth```,but when is an option expired ? Is it when the current date ( today's date) is after the ```lastTradeDateOrContractMonth``` ?
    * ✅ I understand the calculation behind **Post Expiry Market Value**, I am doing the calcualtion but how is it relevant ? For example when it comes to P&L, I understand that if you bought(market Price) at a price higher than the Cost Price, then you bought it more expensive which is represented by a positive value, on the other hand if you bought at a price lower than cost price, you actaully bought it cheaper, you made a profit which is represented by a negative value
         

## 🆕 Testing and Improvements ##

* **Testing and Improvements:**
    1. `ConId` column is not required, unnessary information
        * *Creating copy with dropped`ConId` column*✅
    
    2. No visual indication for option getting expired or excercised other than zeros
        * *Added `Excercised / Expired` column in options portfolio*✅
    
    3. Getting `no Security Found Error` for some stocks for which underlying market equity was to be determined
        * *For getting underlying equity market price, the Stock that is created now has more details*✅
        
    4. Last time while testing the code on zoom, it was observed that the connection was failing, further investigation reveals poor interent connection on my part, thus the disconnection which ultimately leads to further errors if not resolved immediately
        * *In the try except block, we can add `exit` in the except part*✅

    
    

In [ ]:
#Imports
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from sys import exit

#Import ib_insync library
from ib_insync import *

#Instantiate IB class and use .connect() method on it, if TWS is not running ConnectionRefusedError will be raised
ib = IB()

#Only used in interactive environments such as Jupyter Notebooks
util.startLoop()
try:
    print(ib.connect(clientId=0))
except:
    exit()

In [ ]:
#Automatically added to portfolio in TWS, get portfolio
portfolio = ib.portfolio()
df = util.df(portfolio)

In [ ]:
#Extract contracts from portfolio
column = df['contract']
market_values = df['marketValue'].values

contract_types = []
tickers = []
pc = []
strikes = []
exchanges = []
expiry = []
exchanges = []
conIds = []

#Extract the ticker,type of contract and exchange
for obj in column:
    if obj.__class__.__name__ == 'Stock':
        tickers.append(obj.symbol)
        contract_types.append(obj.__class__.__name__)
        exchanges.append(obj.primaryExchange)
        conIds.append(obj.conId)
    if obj.__class__.__name__ == 'Option':
        tickers.append(obj.symbol)
        contract_types.append(obj.__class__.__name__)
        pc.append(obj.right)
        strikes.append(obj.strike)
        exchanges.append(obj.primaryExchange)
        expiry.append(datetime.strptime(obj.lastTradeDateOrContractMonth,"%Y%m%d"))
        conIds.append(obj.conId)

In [ ]:
#Muniplate portfolio, drop uneeded columns
df = df.drop(columns=['contract','realizedPNL','account'])

In [ ]:
#Rename columns
df = df.rename(columns={"position": "Qty", "marketPrice": "Market Price", "averageCost":"Cost Price" , "unrealizedPNL":"P&L", "marketValue": "Total Market Value"})

#Add Type and Ticker columns
df['Type'] = contract_types
df['Ticker'] = tickers
df['Exchange'] = exchanges
df['ConId'] = conIds

#Rearrange columns
df = df[["Type","Ticker","Exchange","ConId","Qty", "Market Price", "Cost Price", "P&L", "Total Market Value"]]

In [ ]:
#Today's date
today = datetime.now()


#Group by Type, either Stock or Option
grouped = df.groupby('Type')

#Create empty dataframe for options and stocks'
df_stocks = pd.DataFrame(np.nan, index=[0],columns=["Type","Ticker","Exchange","ConId","Qty", "Market Price", "Cost Price", "P&L", "Total Market Value"])
df_options = pd.DataFrame(np.nan,index=[0],columns=["Type","Ticker","Exchange","ConId","Qty", "Market Price", "Cost Price", "P&L", "Total Market Value"])

#Split Dataframe into two, one dataframe for stocks, one for options
for commodity in list(grouped.groups):
    if commodity == 'Stock':
        df_stocks = grouped.get_group('Stock')
    elif commodity == 'Option':
        df_options = grouped.get_group('Option')
        
        
df_stocks = df_stocks.drop(columns=['Type'])
df_options = df_options.drop(columns=['Type'])
df_options = df_options.drop(columns=['Total Market Value'])



total_equities_market_value = total_options_market_value = 0
total_pl_stocks = total_pl_options = 0 



In [ ]:
#Get Account details 
details = ib.accountSummary()
details_df = util.df(details)

#Tag Values
#tags = details_df['tag'].values.tolist()
#print(tags)

#Set tag values as index
details_df = details_df.set_index('tag')
net_liquidation_value = int(float(details_df.loc['NetLiquidation','value']))


In [ ]:
if df_stocks['Ticker'].isnull().values.any() == False:
    total_stocks_market_value = df_stocks['Total Market Value'].sum() 
    total_long_stocks_value = df_stocks.loc[df_stocks['Total Market Value'] > 0, 'Total Market Value'].sum()
    total_short_stocks_value = df_stocks.loc[df_stocks['Total Market Value'] < 0, 'Total Market Value'].sum()
    total_stocks_value = total_long_stocks_value + total_short_stocks_value
    total_stocks_absolute_value = abs(total_long_stocks_value) + abs(total_short_stocks_value)
    df_stocks['% Market Value'] = (df_stocks['Total Market Value'] / net_liquidation_value) * 100
    total_pl_stocks = df_stocks['P&L'].sum()
    

In [ ]:
if df_options['Ticker'].isnull().values.any() == False:
    df_options['Total Market Value'] = df_options['Qty'] * df_options['Market Price'] 
    total_options_market_value = df_options['Total Market Value'].sum() 
    total_long_options_value = df_options.loc[df_options['Total Market Value'] > 0, 'Total Market Value'].sum()
    total_short_options_value = df_options.loc[df_options['Total Market Value'] < 0, 'Total Market Value'].sum()
    total_options_value = total_long_options_value + total_short_options_value
    total_options_absolute_value = abs(total_long_options_value) + abs(total_short_options_value)

    df_options['% Market Value'] = (df_options['Total Market Value'] / net_liquidation_value) * 100
    total_pl_options = df_options['P&L'].sum()
    

    #Get underlying market value for option equity
    underlying_market_equitys = []
    for conId in df_options['ConId']:
        #Create contract, for each symbol
        contract = Contract(conId=conId)
        #Qualify the contract
        ib.qualifyContracts(contract)
        #Request ticker and market price
        [ticker] = ib.reqTickers(contract)
        underlying_market_equitys.append(ticker.bidGreeks.undPrice)
        

    #Add column for underlying market equity
    df_options['Underlying Equity Market Price'] = underlying_market_equitys

    #Create Call/Put, Strike Column, Expiry for Options
    df_options['Type'] = pc
    df_options['Strike'] = strikes
    df_options['Expiry'] = expiry

    #Determine post expiry market value, wether it is expired or will it get excercised, if expired option value is zero, if for put underlying equity price is greater than strike then zero, if for call underlying equity is less than strike than zero
    post_expiry_market_values = []
    excercised_or_expired = []
    
    types_option = df_options['Type']
    expiries_option = df_options['Expiry']
    underlyingEquityMarketPrices_option = df_options['Underlying Equity Market Price'] 
    strikes_option = df_options['Strike']
    qtys_option = df_options['Qty']
    
    for Type, Expiry, UnderlyingEquityMarketPrice, Strike, Qty in zip(types_option,expiries_option,underlyingEquityMarketPrices_option,strikes_option,qtys_option):
        if Type == 'P':
            if today < Expiry and UnderlyingEquityMarketPrice < Strike:
                post_expiry_market_value = (Strike - UnderlyingEquityMarketPrice) * Qty * 100
                excercised_or_expired.append('Excercised')
            else:
                
                post_expiry_market_value = 0
                excercised_or_expired.append('Expired')
        else:
            if today < Expiry and UnderlyingEquityMarketPrice > Strike:
                post_expiry_market_value = (UnderlyingEquityMarketPrice - Strike) * Qty * 100
                excercised_or_expired.append('Excercised')
            else:
                post_expiry_market_value = 0
                excercised_or_expired.append('Expired')
        
        post_expiry_market_values.append(post_expiry_market_value)

    df_options['Total Market Value Post Expiry'] = post_expiry_market_values
    df_options['Excercised/Expired'] = excercised_or_expired
    df_options = df_options.fillna('')
    
    total_long_options_value_pe = df_options.loc[df_options['Total Market Value Post Expiry'] > 0, 'Total Market Value Post Expiry'].sum()
    total_short_options_value_pe = df_options.loc[df_options['Total Market Value Post Expiry'] < 0, 'Total Market Value Post Expiry'].sum()
    total_options_value_pe = total_long_options_value_pe + total_short_options_value_pe
    total_options_absolute_value_pe = abs(total_long_options_value_pe) + abs(total_short_options_value_pe)



In [ ]:
#Determine total P&L, percent market value for stocks and options
total_pl = total_pl_stocks + total_pl_options 

percent_equities =  (total_stocks_market_value / net_liquidation_value) * 100
percent_options = (total_options_market_value / net_liquidation_value) * 100

#Create rows for total market value and percentages
row_equities = pd.Series({'P&L':total_pl_stocks,'Total Market Value':total_stocks_market_value, '% Market Value':percent_equities }, name='Total')
row_options = pd.Series({'P&L':total_pl_options,'Total Market Value':total_options_market_value, '% Market Value':percent_options}, name='Total')


#Append rows to stock and options portfolio
df_stocks = df_stocks.append(row_equities, ignore_index=False)
df_options = df_options.append(row_options, ignore_index=False)

#Fill nan value in total row
df_stocks.iloc[-1] = df_stocks.iloc[-1].fillna('')
df_options.iloc[-1] = df_options.iloc[-1].fillna('')



In [ ]:
#Create totals dataframe

#Stock Totals
df_stocks_total = pd.DataFrame(data=[total_long_stocks_value,total_short_stocks_value,total_long_stocks_value + total_short_stocks_value,abs(total_long_stocks_value) + abs(total_short_stocks_value)], columns=['Value'], index=['Total Long Equities','Total Short Equities','Total Equities', 'Total Absolute Equities'])
 

#Option Totals
df_options_total = pd.DataFrame(data=[total_long_options_value, total_short_options_value,total_long_options_value + total_short_options_value, abs(total_long_options_value) + abs(total_short_options_value)], columns=['Value'], index=['Total Long Options','Total Short Options','Total Options', 'Total Absolute Options'])             


#Option Post Expiry Totals
df_options_pe_total = pd.DataFrame(data=[total_long_options_value_pe, total_short_options_value_pe,total_long_options_value_pe + total_short_options_value_pe, abs(total_long_options_value_pe) + abs(total_short_options_value_pe)], columns=['Value'], index=['Total Long Options Post Expiry','Total Short Options Post Expiry','Total Options Post Expiry', 'Total Absolute Options Post Expiry'])             

#Combined Total
combined_total_long_market_value = int(float(df_stocks_total.loc['Total Long Equities', 'Value'])) + int(float(df_options_total.loc['Total Long Options', 'Value']))
combined_total_short_market_value = int(float(df_stocks_total.loc['Total Short Equities', 'Value'])) + int(float(df_options_total.loc['Total Short Options', 'Value']))
combined_total_market_value = int(float(df_stocks_total.loc['Total Equities', 'Value'])) + int(float(df_options_total.loc['Total Options', 'Value']))
combined_total_absolute_market_value = int(float(df_stocks_total.loc['Total Absolute Equities', 'Value'])) + int(float(df_options_total.loc['Total Absolute Options', 'Value']))
combined_total_pe_market_value = int(float(df_stocks_total.loc['Total Equities', 'Value'])) + int(float(df_options_pe_total.loc['Total Options Post Expiry', 'Value']))
combined_total_pe_ab_market_value = int(float(df_stocks_total.loc['Total Absolute Equities', 'Value'])) + int(float(df_options_pe_total.loc['Total Absolute Options Post Expiry', 'Value']))


df_combined_totals = pd.DataFrame(data=[combined_total_long_market_value,combined_total_short_market_value,combined_total_market_value,combined_total_absolute_market_value,combined_total_pe_market_value,combined_total_pe_ab_market_value],columns=['Value'], index=['Combined Total Long Market Value','Combined Total Short Market Value','Combined Total Market Value','Combined Total Absolute Market Value','Combined Total Post-Expiry Market Value','Combined Total Post-Expiry Absolute Value'])


In [ ]:
#Display results
df_stocks_displayable = df_stocks.drop(['ConId'], axis=1)
display(df_stocks_displayable.style.set_caption("Stocks Portfolio"))

df_options_displayable = df_options.drop(['ConId'], axis=1)
display(df_options_displayable.style.set_caption("Options Portfolio"))

display(df_stocks_total.style.set_caption('Total Equities'))
display(df_options_total.style.set_caption('Total Options'))
display(df_options_pe_total.style.set_caption('Total Options Post Expiry'))
display(df_combined_totals.style.set_caption('Total Combined'))


In [ ]:
ib.disconnect()

In [ ]:
#Store results to be used in Margins.ipynb
%store df_stocks
%store df_options